In [ ]:
import os
import random
import shutil

def split_dataset(dataset_path, part1_path, part2_path):
    # Create directories if they don't exist
    os.makedirs(part1_path, exist_ok=True)
    os.makedirs(part2_path, exist_ok=True)

    # List all images in the dataset
    images = [f for f in os.listdir(dataset_path) if os.path.isfile(os.path.join(dataset_path, f))]
    random.shuffle(images)

    # Split the dataset into two equal parts
    split_index = len(images) // 2
    part1_images = images[:split_index]
    part2_images = images[split_index:]

    # Move images to their respective directories
    for img in part1_images:
        shutil.move(os.path.join(dataset_path, img), os.path.join(part1_path, img))
    for img in part2_images:
        shutil.move(os.path.join(dataset_path, img), os.path.join(part2_path, img))

    print(f"Dataset split into {part1_path} and {part2_path}")

# Example usage
split_dataset('Datasets\places2', 'Datasets\split1', 'Datasets/split2')

Dataset split into Github/inpainting_and_forgery_detection/split1 and Github/inpainting_and_forgery_detection/split2


In [5]:
%pip install torch torchvision torchaudio


  Using cached torch-2.6.0-cp312-cp312-win_amd64.whl.metadata (28 kB)
  Using cached torchvision-0.21.0-cp312-cp312-win_amd64.whl.metadata (6.3 kB)
  Using cached torchaudio-2.6.0-cp312-cp312-win_amd64.whl.metadata (6.7 kB)
  Using cached filelock-3.18.0-py3-none-any.whl.metadata (2.9 kB)
  Using cached networkx-3.4.2-py3-none-any.whl.metadata (6.3 kB)
  Using cached jinja2-3.1.6-py3-none-any.whl.metadata (2.9 kB)
  Using cached fsspec-2025.3.0-py3-none-any.whl.metadata (11 kB)
  Using cached sympy-1.13.1-py3-none-any.whl.metadata (12 kB)
  Using cached mpmath-1.3.0-py3-none-any.whl.metadata (8.6 kB)
  Using cached numpy-2.2.4-cp312-cp312-win_amd64.whl.metadata (60 kB)
  Using cached pillow-11.1.0-cp312-cp312-win_amd64.whl.metadata (9.3 kB)
  Using cached MarkupSafe-3.0.2-cp312-cp312-win_amd64.whl.metadata (4.1 kB)
Using cached torch-2.6.0-cp312-cp312-win_amd64.whl (204.1 MB)
Using cached sympy-1.13.1-py3-none-any.whl (6.2 MB)
Using cached torchvision-0.21.0-cp312-cp312-win_amd64.whl (


[notice] A new release of pip is available: 24.3.1 -> 25.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [ ]:
pip install matplotlib pandas numpy 


  Using cached matplotlib-3.10.1-cp312-cp312-win_amd64.whl.metadata (11 kB)
  Using cached pandas-2.2.3-cp312-cp312-win_amd64.whl.metadata (19 kB)
  Using cached contourpy-1.3.1-cp312-cp312-win_amd64.whl.metadata (5.4 kB)
  Using cached cycler-0.12.1-py3-none-any.whl.metadata (3.8 kB)
  Using cached fonttools-4.56.0-cp312-cp312-win_amd64.whl.metadata (103 kB)
  Using cached kiwisolver-1.4.8-cp312-cp312-win_amd64.whl.metadata (6.3 kB)
  Using cached tzdata-2025.2-py2.py3-none-any.whl.metadata (1.4 kB)
Using cached matplotlib-3.10.1-cp312-cp312-win_amd64.whl (8.1 MB)
Using cached pandas-2.2.3-cp312-cp312-win_amd64.whl (11.5 MB)
Using cached contourpy-1.3.1-cp312-cp312-win_amd64.whl (220 kB)
Using cached cycler-0.12.1-py3-none-any.whl (8.3 kB)
Using cached fonttools-4.56.0-cp312-cp312-win_amd64.whl (2.2 MB)
Using cached kiwisolver-1.4.8-cp312-cp312-win_amd64.whl (71 kB)
Using cached tzdata-2025.2-py2.py3-none-any.whl (347 kB)
Note: you may need to restart the kernel to use updated package


[notice] A new release of pip is available: 24.3.1 -> 25.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [4]:
%pip install opencv-python


   ---------------------------------------- 0.0/39.5 MB ? eta -:--:--
   -- ------------------------------------- 2.9/39.5 MB 14.0 MB/s eta 0:00:03
   ------ --------------------------------- 6.6/39.5 MB 15.5 MB/s eta 0:00:03
   --------- ------------------------------ 9.4/39.5 MB 15.1 MB/s eta 0:00:02
   ------------ --------------------------- 12.3/39.5 MB 14.3 MB/s eta 0:00:02
   --------------- ------------------------ 15.2/39.5 MB 14.3 MB/s eta 0:00:02
   ------------------ --------------------- 17.8/39.5 MB 13.9 MB/s eta 0:00:02
   -------------------- ------------------- 20.7/39.5 MB 13.8 MB/s eta 0:00:02
   ----------------------- ---------------- 23.3/39.5 MB 13.7 MB/s eta 0:00:02
   -------------------------- ------------- 26.2/39.5 MB 13.6 MB/s eta 0:00:01
   ----------------------------- ---------- 28.8/39.5 MB 13.5 MB/s eta 0:00:01
   ------------------------------- -------- 31.5/39.5 MB 13.5 MB/s eta 0:00:01
   ---------------------------------- ----- 34.3/39.5 MB 13.5 MB


[notice] A new release of pip is available: 24.3.1 -> 25.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [6]:
%pip show torch

Name: torch
Version: 2.6.0
Summary: Tensors and Dynamic neural networks in Python with strong GPU acceleration
Home-page: https://pytorch.org/
Author: PyTorch Team
Author-email: packages@pytorch.org
License: BSD-3-Clause
Location: c:\Users\revna\Desktop\Github\Inpainting_and_Forgery_Detection\venv\Lib\site-packages
Requires: filelock, fsspec, jinja2, networkx, setuptools, sympy, typing-extensions
Required-by: torchaudio, torchvision
Note: you may need to restart the kernel to use updated packages.


In [11]:
!python -m ipykernel install --user --name=your_env --display-name "Python (venv)"


Installed kernelspec your_env in C:\Users\revna\AppData\Roaming\jupyter\kernels\your_env


In [1]:
import torch
print(torch.__version__)

2.6.0+cpu


In [2]:
import torch
import torchvision
import PIL
import numpy
import matplotlib.pyplot
print("All imports work!")

All imports work!


In [3]:
%pip install tqdm

  Using cached tqdm-4.67.1-py3-none-any.whl.metadata (57 kB)
Using cached tqdm-4.67.1-py3-none-any.whl (78 kB)
Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.3.1 -> 25.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [ ]:
import cv2
import numpy as np
import os
import matplotlib.pyplot as plt
from tqdm import tqdm  # For progress tracking

def create_mask(image_shape, min_mask_percentage=0.10, max_mask_percentage=0.30):
    """
    Create a more dynamic mask with controlled total masked area between min and max percentages
    
    Args:
    - image_shape: Shape of the image (height, width)
    - min_mask_percentage: Minimum percentage of image to be masked (default 10%)
    - max_mask_percentage: Maximum percentage of image to be masked (default 30%)
    
    Returns:
    - mask: Numpy array with mask (255 for masked regions)
    """
    height, width = image_shape[:2]
    total_pixels = height * width
    
    # Ensure min is not greater than max
    min_mask_percentage = min(min_mask_percentage, max_mask_percentage)
    
    # Calculate target masked pixels within the range
    min_masked_pixels = int(total_pixels * min_mask_percentage)
    max_masked_pixels = int(total_pixels * max_mask_percentage)
    target_masked_pixels = np.random.randint(min_masked_pixels, max_masked_pixels + 1)
    
    # Initialize mask
    mask = np.zeros((height, width), dtype=np.uint8)
    
    # Randomize mask generation approach
    mask_type = np.random.choice(['rectangle', 'multiple', 'irregular'])
    
    if mask_type == 'rectangle':
        # Single rectangular region
        max_side = int(np.sqrt(target_masked_pixels))
        rect_width = np.random.randint(max_side // 2, max_side)
        rect_height = target_masked_pixels // rect_width
        
        x = np.random.randint(0, width - rect_width)
        y = np.random.randint(0, height - rect_height)
        
        mask[y:y+rect_height, x:x+rect_width] = 255
    
    elif mask_type == 'multiple':
        # Multiple smaller masked regions
        num_regions = np.random.randint(2, 6)
        pixels_per_region = target_masked_pixels // num_regions
        
        for _ in range(num_regions):
            region_size = int(np.sqrt(pixels_per_region))
            x = np.random.randint(0, width - region_size)
            y = np.random.randint(0, height - region_size)
            
            mask[y:y+region_size, x:x+region_size] = 255
    
    else:  # irregular mask
        # Create an irregular mask using random walk
        current_masked_pixels = 0
        max_attempts = 1000
        attempts = 0
        
        while current_masked_pixels < target_masked_pixels and attempts < max_attempts:
            # Random walk
            x, y = np.random.randint(0, width), np.random.randint(0, height)
            step_size = np.random.randint(1, 10)
            
            # Ensure we don't go out of bounds
            x = max(0, min(x, width-1))
            y = max(0, min(y, height-1))
            
            # Create a small irregular region
            region = mask[max(0, y-step_size):min(height, y+step_size),
                          max(0, x-step_size):min(width, x+step_size)]
            
            # Add region to mask if not already masked
            new_mask_pixels = np.sum(region == 0)
            if new_mask_pixels > 0:
                region[region == 0] = 255
                current_masked_pixels += new_mask_pixels
            
            attempts += 1
    
    # Verify mask percentage (optional - can be commented out for bulk processing)
    # masked_percentage = np.sum(mask == 255) / total_pixels
    # print(f"Mask Type: {mask_type}, Masked Percentage: {masked_percentage:.2%}, " 
    #       f"Range: {min_mask_percentage:.2%}-{max_mask_percentage:.2%}")
    
    return mask


def inpaint_image(image_path, output_dir, min_mask_percentage=0.05, max_mask_percentage=0.15, save_visualization=False):
    """
    Inpaint an image using OpenCV's Telea inpainting method
    
    Args:
    - image_path: Path to input image
    - output_dir: Directory to save output images
    - min_mask_percentage: Minimum percentage of image to be masked
    - max_mask_percentage: Maximum percentage of image to be masked
    - save_visualization: Whether to save the visualization (False for bulk processing)
    """
    # Create output directories
    os.makedirs(output_dir, exist_ok=True)
    if save_visualization:
        os.makedirs(os.path.join(output_dir, 'visualizations'), exist_ok=True)
    
    # Read the image
    image = cv2.imread(image_path)
    if image is None:
        print(f"Warning: Couldn't read image {image_path}")
        return
    
    # Create a mask with specified min/max percentage
    mask = create_mask(image.shape, min_mask_percentage, max_mask_percentage)
    
    # Telea Method
    inpainted_telea = cv2.inpaint(image, mask, inpaintRadius=3, flags=cv2.INPAINT_TELEA)
    
    # Save inpainted image
    filename = os.path.basename(image_path)
    cv2.imwrite(os.path.join(output_dir, f'telea_{filename}'), inpainted_telea)
    
    # Save visualization only if requested (disabled for bulk processing)
    if save_visualization:
        # Create visualization figure
        plt.figure(figsize=(15, 5))
        
        # Original Image
        plt.subplot(1, 3, 1)
        plt.title('Original Image')
        plt.imshow(cv2.cvtColor(image, cv2.COLOR_BGR2RGB))
        plt.axis('off')
        
        # Mask Visualization
        plt.subplot(1, 3, 2)
        plt.title('Mask')
        plt.imshow(mask, cmap='gray')
        plt.axis('off')
        
        # Telea Inpainting
        plt.subplot(1, 3, 3)
        plt.title('Telea Inpainting')
        plt.imshow(cv2.cvtColor(inpainted_telea, cv2.COLOR_BGR2RGB))
        plt.axis('off')
        
        # Save visualization
        vis_path = os.path.join(output_dir, 'visualizations', f'inpainting_{filename}')
        plt.tight_layout()
        plt.savefig(vis_path)
        plt.close()


def process_directory(input_dir, output_dir, max_images=10000, min_mask_percentage=0.05, max_mask_percentage=0.15):
    """
    Process up to max_images images in a directory
    
    Args:
    - input_dir: Directory with input images
    - output_dir: Directory to save output images
    - max_images: Maximum number of images to process
    - min_mask_percentage: Minimum percentage of image to be masked
    - max_mask_percentage: Maximum percentage of image to be masked
    """
    # Ensure output directory exists
    os.makedirs(output_dir, exist_ok=True)
    
    # Get list of all image files
    image_files = []
    for root, _, files in os.walk(input_dir):
        for filename in files:
            if filename.lower().endswith(('.png', '.jpg', '.jpeg', '.bmp', '.tiff')):
                image_files.append(os.path.join(root, filename))
    
    # Limit to max_images
    total_images = min(len(image_files), max_images)
    if len(image_files) < max_images:
        print(f"Warning: Only found {len(image_files)} images, fewer than the requested {max_images}")
    else:
        print(f"Processing {total_images} images...")
    
    # Process images with progress bar
    for image_path in tqdm(image_files[:total_images], desc="Inpainting Images"):
        try:
            # Disable visualization for bulk processing to speed up
            inpaint_image(
                image_path=image_path,
                output_dir=output_dir,
                min_mask_percentage=min_mask_percentage,
                max_mask_percentage=max_mask_percentage,
                save_visualization=True  
            )
        except Exception as e:
            print(f"Error processing {image_path}: {str(e)}")


def main():
    # Process images
    process_directory(
        input_dir='Datasets/split2',  # Input image directory
        output_dir='Datasets/in_split2',  # Output directory
        max_images=5000,  # Process up to 5,000 images
        min_mask_percentage=0.05,  # Minimum 5% of image will be masked
        max_mask_percentage=0.15   # Maximum 15% of image will be masked
    )

if __name__ == '__main__':
    main()

Processing 5000 images...


Inpainting Images: 100%|██████████| 5000/5000 [20:46<00:00,  4.01it/s]


In [1]:
import os
import random
import shutil

def label_and_shuffle(part1_dir, part2_dir, output_dir, part1_limit):
    os.makedirs(output_dir, exist_ok=True)
    
    # Get images from part1 with limit
    part1_images = os.listdir(part1_dir)
    if len(part1_images) > part1_limit:
        part1_images = part1_images[:part1_limit]
    
    labeled_images = []
    for img_name in part1_images:
        labeled_images.append((os.path.join(part1_dir, img_name), 0))
    
    # Get all images from part2
    for img_name in os.listdir(part2_dir):
        labeled_images.append((os.path.join(part2_dir, img_name), 1))
    
    random.shuffle(labeled_images)
    
    for i, (img_path, label) in enumerate(labeled_images):
        img_name = f"{i}_{label}.png"
        shutil.copy(img_path, os.path.join(output_dir, img_name))
    
    print(f"Dataset labeled and shuffled - Used {len(part1_images)} images from part1 and {len(os.listdir(part2_dir))} from part2")

# Example usage
label_and_shuffle('Datasets/split1', 'Datasets/in_split2', 'Datasets/shuffle_inpai_data', 5000)

Dataset labeled and shuffled - Used 5000 images from part1 and 5000 from part2


In [2]:
import os
import random
import shutil

def split_train_test(shuffled_dataset_dir, train_dir, test_dir):
    os.makedirs(train_dir, exist_ok=True)
    os.makedirs(test_dir, exist_ok=True)
    
    images = [f for f in os.listdir(shuffled_dataset_dir) if os.path.isfile(os.path.join(shuffled_dataset_dir, f))]
    random.shuffle(images)
    
    split_index = len(images) // 2
    train_images = images[:split_index]
    test_images = images[split_index:]
    
    for img in train_images:
        shutil.move(os.path.join(shuffled_dataset_dir, img), os.path.join(train_dir, img))
    for img in test_images:
        shutil.move(os.path.join(shuffled_dataset_dir, img), os.path.join(test_dir, img))
    
    print(f"Dataset split into training and testing sets")

# Example usage
split_train_test('Datasets/shuffle_inpai_data', 'Datasets/norm_train', 'Datasets/norm_test')

Dataset split into training and testing sets


In [3]:
%pip install keras

   ---------------------------------------- 0.0/1.3 MB ? eta -:--:--
   ---------------------------------------- 0.0/1.3 MB ? eta -:--:--
   ------- -------------------------------- 0.3/1.3 MB ? eta -:--:--
   ------- -------------------------------- 0.3/1.3 MB ? eta -:--:--
   ------------------------------- -------- 1.0/1.3 MB 1.4 MB/s eta 0:00:01
   ---------------------------------------- 1.3/1.3 MB 1.6 MB/s eta 0:00:00
   ---------------------------------------- 0.0/3.0 MB ? eta -:--:--
   ---------------------------------------- 0.0/3.0 MB ? eta -:--:--
   --- ------------------------------------ 0.3/3.0 MB ? eta -:--:--
   ------- -------------------------------- 0.5/3.0 MB 837.5 kB/s eta 0:00:03
   -------------- ------------------------- 1.0/3.0 MB 1.3 MB/s eta 0:00:02
   --------------------- ------------------ 1.6/3.0 MB 1.6 MB/s eta 0:00:01
   ---------------------------- ----------- 2.1/3.0 MB 2.0 MB/s eta 0:00:01
   ------------------------------- -------- 2.4/3.0 MB 2.0 


[notice] A new release of pip is available: 24.3.1 -> 25.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [4]:
%pip install scikit-learn

  Using cached scikit_learn-1.6.1-cp312-cp312-win_amd64.whl.metadata (15 kB)
  Using cached scipy-1.15.2-cp312-cp312-win_amd64.whl.metadata (60 kB)
  Using cached joblib-1.4.2-py3-none-any.whl.metadata (5.4 kB)
  Using cached threadpoolctl-3.6.0-py3-none-any.whl.metadata (13 kB)
Using cached scikit_learn-1.6.1-cp312-cp312-win_amd64.whl (11.1 MB)
Using cached joblib-1.4.2-py3-none-any.whl (301 kB)
Using cached scipy-1.15.2-cp312-cp312-win_amd64.whl (40.9 MB)
Using cached threadpoolctl-3.6.0-py3-none-any.whl (18 kB)
Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.3.1 -> 25.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [5]:
%pip install h5py

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.3.1 -> 25.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [6]:
import os
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms
from PIL import Image
import matplotlib.pyplot as plt
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report
import random
import glob

# Set random seeds for reproducibility
random_seed = 42
torch.manual_seed(random_seed)
torch.cuda.manual_seed(random_seed)
np.random.seed(random_seed)
random.seed(random_seed)
torch.backends.cudnn.deterministic = True

# Device configuration
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Using device: {device}")

# Custom Dataset class
class ImageForgeryDataset(Dataset):
    def __init__(self, image_dir, transform=None):
        """
        Args:
            image_dir (string): Directory with all the images and their corresponding labels
            transform (callable, optional): Optional transform to be applied on a sample
        """
        self.image_dir = image_dir
        self.transform = transform
        
        # Get all image files
        self.image_paths = []
        for ext in ['*.jpg', '*.jpeg', '*.png']:
            self.image_paths.extend(glob.glob(os.path.join(image_dir, ext)))
        
        print(f"Found {len(self.image_paths)} images in {image_dir}")
    
    def __len__(self):
        return len(self.image_paths)
    
    def __getitem__(self, idx):
        image_path = self.image_paths[idx]
        image = Image.open(image_path).convert('RGB')
        
        # Extract label from filename - assuming filename ends with _0.jpg or _1.jpg
        # where 0 = authentic, 1 = forged
        filename = os.path.basename(image_path)
        label = int(filename.split('_')[-1].split('.')[0])
        
        if self.transform:
            image = self.transform(image)
        
        return image, label

# Define transformations
transform = transforms.Compose([
    transforms.Resize((256, 256)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

# Define the Generator
class Generator(nn.Module):
    def __init__(self, latent_dim=100, channels=3):
        super(Generator, self).__init__()
        self.latent_dim = latent_dim
        
        self.model = nn.Sequential(
            # Initial block
            nn.ConvTranspose2d(latent_dim, 512, 4, 1, 0, bias=False),
            nn.BatchNorm2d(512),
            nn.ReLU(True),
            
            # Second block
            nn.ConvTranspose2d(512, 256, 4, 2, 1, bias=False),
            nn.BatchNorm2d(256),
            nn.ReLU(True),
            
            # Third block
            nn.ConvTranspose2d(256, 128, 4, 2, 1, bias=False),
            nn.BatchNorm2d(128),
            nn.ReLU(True),
            
            # Fourth block
            nn.ConvTranspose2d(128, 64, 4, 2, 1, bias=False),
            nn.BatchNorm2d(64),
            nn.ReLU(True),
            
            # Output block
            nn.ConvTranspose2d(64, channels, 4, 2, 1, bias=False),
            nn.Tanh()
        )

    def forward(self, z):
        z = z.view(z.size(0), self.latent_dim, 1, 1)
        return self.model(z)

# Define the Discriminator
class Discriminator(nn.Module):
    def __init__(self, channels=3):
        super(Discriminator, self).__init__()
        
        self.feature_extractor = nn.Sequential(
            # First block
            nn.Conv2d(channels, 64, 4, 2, 1, bias=False),
            nn.LeakyReLU(0.2, inplace=True),
            
            # Second block
            nn.Conv2d(64, 128, 4, 2, 1, bias=False),
            nn.BatchNorm2d(128),
            nn.LeakyReLU(0.2, inplace=True),
            
            # Third block
            nn.Conv2d(128, 256, 4, 2, 1, bias=False),
            nn.BatchNorm2d(256),
            nn.LeakyReLU(0.2, inplace=True),
            
            # Fourth block
            nn.Conv2d(256, 512, 4, 2, 1, bias=False),
            nn.BatchNorm2d(512),
            nn.LeakyReLU(0.2, inplace=True),
        )
        
        # Output layer for GAN discrimination (real/fake)
        self.gan_output = nn.Sequential(
            nn.Conv2d(512, 1, 4, 1, 0, bias=False),
            nn.Sigmoid()
        )
        
        # Output layer for classification (authentic/forged)
        self.classifier = nn.Sequential(
            nn.Flatten(),
            nn.Linear(512 * 16 * 16, 512),  # Adjusted based on feature map size
            nn.LeakyReLU(0.2, inplace=True),
            nn.Dropout(0.3),
            nn.Linear(512, 1),
            nn.Sigmoid()
        )
        
    def forward(self, img):
        features = self.feature_extractor(img)
        validity = self.gan_output(features)
        classification = self.classifier(features)
        return validity.view(-1), classification.view(-1)

# Training function
def train_gan_classifier(generator, discriminator, dataloader, epochs=50, latent_dim=100, 
                         lr_g=0.0002, lr_d=0.0002, betas=(0.5, 0.999), save_path='model'):
    
    # Loss functions
    adversarial_loss = nn.BCELoss()
    classification_loss = nn.BCELoss()
    
    # Optimizers
    optimizer_G = optim.Adam(generator.parameters(), lr=lr_g, betas=betas)
    optimizer_D = optim.Adam(discriminator.parameters(), lr=lr_d, betas=betas)
    
    # Training loop
    for epoch in range(epochs):
        for i, (real_imgs, labels) in enumerate(dataloader):
            
            batch_size = real_imgs.size(0)
            real_imgs = real_imgs.to(device)
            labels = labels.float().to(device)
            
            # Ground truths
            valid = torch.ones(batch_size, 1, requires_grad=False).to(device)
            fake = torch.zeros(batch_size, 1, requires_grad=False).to(device)
            
            # -----------------
            #  Train Generator
            # -----------------
            optimizer_G.zero_grad()
            
            # Sample noise as generator input
            z = torch.randn(batch_size, latent_dim).to(device)
            
            # Generate fake images
            gen_imgs = generator(z)
            
            # Calculate loss and backpropagate
            validity, _ = discriminator(gen_imgs)
            g_loss = adversarial_loss(validity, valid)
            
            g_loss.backward()
            optimizer_G.step()
            
            # -------------------
            #  Train Discriminator
            # -------------------
            optimizer_D.zero_grad()
            
            # Compute loss for real images
            real_validity, real_pred = discriminator(real_imgs)
            d_real_gan_loss = adversarial_loss(real_validity, valid)
            d_real_class_loss = classification_loss(real_pred, labels)
            
            # Compute loss for fake images
            fake_validity, _ = discriminator(gen_imgs.detach())
            d_fake_gan_loss = adversarial_loss(fake_validity, fake)
            
            # Total discriminator loss
            d_loss = d_real_gan_loss + d_fake_gan_loss + d_real_class_loss
            
            d_loss.backward()
            optimizer_D.step()
            
            # Print progress
            if i % 10 == 0:
                print(f"[Epoch {epoch}/{epochs}] [Batch {i}/{len(dataloader)}] "
                      f"[D loss: {d_loss.item():.4f}] [G loss: {g_loss.item():.4f}] "
                      f"[Class loss: {d_real_class_loss.item():.4f}]")
        
        # Save models periodically
        if (epoch + 1) % 10 == 0 or epoch == epochs - 1:
            os.makedirs(save_path, exist_ok=True)
            torch.save(generator.state_dict(), f"{save_path}/generator_epoch_{epoch+1}.pth")
            torch.save(discriminator.state_dict(), f"{save_path}/discriminator_epoch_{epoch+1}.pth")
            print(f"Models saved at epoch {epoch+1}")

# Evaluation function
def evaluate_model(discriminator, dataloader):
    discriminator.eval()
    
    all_preds = []
    all_labels = []
    
    with torch.no_grad():
        for imgs, labels in dataloader:
            imgs = imgs.to(device)
            labels = labels.to(device)
            
            _, predictions = discriminator(imgs)
            predictions = (predictions > 0.5).float()
            
            all_preds.extend(predictions.cpu().numpy())
            all_labels.extend(labels.cpu().numpy())
    
    # Calculate accuracy
    accuracy = accuracy_score(all_labels, all_preds)
    print(f"Accuracy: {accuracy * 100:.2f}%")
    
    # Print detailed classification report
    print("\nClassification Report:")
    print(classification_report(all_labels, all_preds, target_names=['Authentic', 'Forged']))
    
    # Generate confusion matrix
    cm = confusion_matrix(all_labels, all_preds)
    plt.figure(figsize=(8, 6))
    plt.imshow(cm, interpolation='nearest', cmap=plt.cm.Blues)
    plt.title('Confusion Matrix')
    plt.colorbar()
    plt.xlabel('Predicted')
    plt.ylabel('Actual')
    tick_marks = [0, 1]
    plt.xticks(tick_marks, ['Authentic', 'Forged'])
    plt.yticks(tick_marks, ['Authentic', 'Forged'])
    
    # Add text annotations
    thresh = cm.max() / 2
    for i in range(cm.shape[0]):
        for j in range(cm.shape[1]):
            plt.text(j, i, format(cm[i, j], 'd'),
                     horizontalalignment="center",
                     color="white" if cm[i, j] > thresh else "black")
    
    plt.tight_layout()
    plt.savefig('confusion_matrix.png')
    plt.show()
    
    return accuracy

# Main execution function
def main():
    # Data directories - update these paths to your data locations
    train_dir = "Datasets/norm_train"
    test_dir = "Datasets/norm_test"
    model_save_path = "Models/norm_det_model"
    
    # Create datasets and dataloaders
    train_dataset = ImageForgeryDataset(train_dir, transform=transform)
    test_dataset = ImageForgeryDataset(test_dir, transform=transform)
    
    train_dataloader = DataLoader(train_dataset, batch_size=32, shuffle=True, num_workers=4)
    test_dataloader = DataLoader(test_dataset, batch_size=32, shuffle=False, num_workers=4)
    
    # Initialize models
    latent_dim = 100
    generator = Generator(latent_dim=latent_dim).to(device)
    discriminator = Discriminator().to(device)
    
    # Train the models
    train_gan_classifier(
        generator=generator,
        discriminator=discriminator,
        dataloader=train_dataloader,
        epochs=50,
        latent_dim=latent_dim,
        save_path=model_save_path
    )
    
    # Load the best model for evaluation (typically the last one)
    last_epoch = 50  # Change if using different number of epochs
    discriminator.load_state_dict(torch.load(f"{model_save_path}/discriminator_epoch_{last_epoch}.pth"))
    
    # Evaluate the model
    accuracy = evaluate_model(discriminator, test_dataloader)
    
    # Save final model in h5 format (for compatibility)
    try:
        import h5py
        torch.save(discriminator.state_dict(), f"{model_save_path}/final_discriminator_model.h5")
        print(f"Model saved in H5 format at {model_save_path}/final_discriminator_model.h5")
    except ImportError:
        print("h5py not installed. Model saved in .pth format only.")
    
    return accuracy

if __name__ == "__main__":
    main()

Using device: cpu
Found 5000 images in Datasets/norm_train
Found 5000 images in Datasets/norm_test


: 

In [ ]:
import os
import numpy as np
import matplotlib.pyplot as plt
from keras.models import Sequential, Model
from keras.layers import Dense, Reshape, Flatten, Conv2D, Conv2DTranspose, LeakyReLU, Dropout, Input
from keras.optimizers import Adam
from keras.datasets import mnist
from keras.utils import to_categorical
from sklearn.model_selection import train_test_split

# Load dataset (replace with your own dataset)
(x_train, y_train), (x_test, y_test) = mnist.load_data()

# Normalize and reshape data
x_train = (x_train.astype(np.float32) - 127.5) / 127.5
x_train = np.expand_dims(x_train, axis=3)
x_test = (x_test.astype(np.float32) - 127.5) / 127.5
x_test = np.expand_dims(x_test, axis=3)

# Split a part of the training data for validation
x_train, x_val, y_train, y_val = train_test_split(x_train, y_train, test_size=0.2, stratify=y_train)

# GAN Parameters
latent_dim = 100
img_shape = x_train.shape[1:]

# Generator
generator = Sequential([
    Dense(128 * 7 * 7, activation="relu", input_dim=latent_dim),
    Reshape((7, 7, 128)),
    Conv2DTranspose(128, kernel_size=4, strides=2, padding='same'),
    LeakyReLU(alpha=0.2),
    Conv2DTranspose(128, kernel_size=4, strides=2, padding='same'),
    LeakyReLU(alpha=0.2),
    Conv2D(1, kernel_size=7, activation='tanh', padding='same')
])

# Discriminator
discriminator = Sequential([
    Conv2D(64, kernel_size=3, strides=2, input_shape=img_shape, padding="same"),
    LeakyReLU(alpha=0.2),
    Dropout(0.3),
    Conv2D(128, kernel_size=3, strides=2, padding="same"),
    LeakyReLU(alpha=0.2),
    Dropout(0.3),
    Flatten(),
    Dense(1, activation='sigmoid')
])

# Compile Discriminator
discriminator.compile(loss='binary_crossentropy', optimizer=Adam(0.0002, 0.5), metrics=['accuracy'])

# Combined model
z = Input(shape=(latent_dim,))
img = generator(z)
discriminator.trainable = False
validity = discriminator(img)
combined = Model(z, validity)
combined.compile(loss='binary_crossentropy', optimizer=Adam(0.0002, 0.5))

# Training the GAN
def train_gan(epochs, batch_size=128, save_interval=50):
    valid = np.ones((batch_size, 1))
    fake = np.zeros((batch_size, 1))

    for epoch in range(epochs):
        # Train Discriminator
        idx = np.random.randint(0, x_train.shape[0], batch_size)
        imgs = x_train[idx]
        noise = np.random.normal(0, 1, (batch_size, latent_dim))
        gen_imgs = generator.predict(noise)
        d_loss_real = discriminator.train_on_batch(imgs, valid)
        d_loss_fake = discriminator.train_on_batch(gen_imgs, fake)
        d_loss = 0.5 * np.add(d_loss_real, d_loss_fake)

        # Train Generator
        noise = np.random.normal(0, 1, (batch_size, latent_dim))
        g_loss = combined.train_on_batch(noise, valid)

        # Save the progress
        if epoch % save_interval == 0:
            print(f"{epoch} [D loss: {d_loss[0]} | D accuracy: {100*d_loss[1]}] [G loss: {g_loss}]")

train_gan(epochs=10000, batch_size=64, save_interval=200)

# Save the generator model
generator.save('gan_generator.h5')

# Generate augmented data
noise = np.random.normal(0, 1, (10000, latent_dim))
gen_imgs = generator.predict(noise)
gen_imgs = (gen_imgs * 127.5 + 127.5).astype(np.uint8)

# Append generated images to original training data
x_train_aug = np.concatenate((x_train, gen_imgs))
y_train_aug = np.concatenate((y_train, np.zeros(10000)))

# Define and train the classifier
classifier = Sequential([
    Conv2D(32, kernel_size=3, activation='relu', input_shape=img_shape),
    MaxPooling2D(pool_size=2),
    Flatten(),
    Dense(128, activation='relu'),
    Dense(1, activation='sigmoid')
])

classifier.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
classifier.fit(x_train_aug, y_train_aug, epochs=10, batch_size=64, validation_data=(x_val, y_val))

# Evaluate the model
loss, accuracy = classifier.evaluate(x_test, y_test)
print(f"Test accuracy: {accuracy*100}%")

# Save the classifier model
classifier.save('image_classifier.h5')

In [ ]:
import numpy as np
import tensorflow as tf
from tensorflow.keras import layers, models, optimizers
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
import matplotlib.pyplot as plt

# Set random seeds for reproducibility
np.random.seed(42)
tf.random.set_seed(42)

class ForgeryDetectionGAN:
    def __init__(self, img_shape=(128, 128, 3), latent_dim=100):
        self.img_shape = img_shape
        self.latent_dim = latent_dim
        
        # Build and compile the discriminator
        self.discriminator = self.build_discriminator()
        self.discriminator.compile(
            loss='binary_crossentropy',
            optimizer=optimizers.Adam(learning_rate=0.0002, beta_1=0.5),
            metrics=['accuracy']
        )
        
        # Build the generator
        self.generator = self.build_generator()
        
        # The generator takes noise as input and generates images
        z = layers.Input(shape=(self.latent_dim,))
        img = self.generator(z)
        
        # For the combined model we will only train the generator
        self.discriminator.trainable = False
        
        # The discriminator takes generated images as input and determines validity
        validity = self.discriminator(img)
        
        # The combined model (stacks generator and discriminator)
        self.combined = models.Model(z, validity)
        self.combined.compile(
            loss='binary_crossentropy',
            optimizer=optimizers.Adam(learning_rate=0.0002, beta_1=0.5)
        )
        
        # For classification
        self.classifier = self.build_classifier()
        self.classifier.compile(
            loss='binary_crossentropy',
            optimizer=optimizers.Adam(learning_rate=0.0002),
            metrics=['accuracy']
        )
    
    def build_generator(self):
        model = models.Sequential()
        
        model.add(layers.Dense(128 * 32 * 32, activation="relu", input_dim=self.latent_dim))
        model.add(layers.Reshape((32, 32, 128)))
        
        model.add(layers.UpSampling2D())
        model.add(layers.Conv2D(128, kernel_size=3, padding="same"))
        model.add(layers.BatchNormalization(momentum=0.8))
        model.add(layers.Activation("relu"))
        
        model.add(layers.UpSampling2D())
        model.add(layers.Conv2D(64, kernel_size=3, padding="same"))
        model.add(layers.BatchNormalization(momentum=0.8))
        model.add(layers.Activation("relu"))
        
        model.add(layers.Conv2D(self.img_shape[2], kernel_size=3, padding="same"))
        model.add(layers.Activation("tanh"))
        
        noise = layers.Input(shape=(self.latent_dim,))
        img = model(noise)
        
        return models.Model(noise, img)
    
    def build_discriminator(self):
        model = models.Sequential()
        
        model.add(layers.Conv2D(32, kernel_size=3, strides=2, input_shape=self.img_shape, padding="same"))
        model.add(layers.LeakyReLU(alpha=0.2))
        model.add(layers.Dropout(0.25))
        
        model.add(layers.Conv2D(64, kernel_size=3, strides=2, padding="same"))
        model.add(layers.BatchNormalization(momentum=0.8))
        model.add(layers.LeakyReLU(alpha=0.2))
        model.add(layers.Dropout(0.25))
        
        model.add(layers.Conv2D(128, kernel_size=3, strides=2, padding="same"))
        model.add(layers.BatchNormalization(momentum=0.8))
        model.add(layers.LeakyReLU(alpha=0.2))
        model.add(layers.Dropout(0.25))
        
        model.add(layers.Flatten())
        model.add(layers.Dense(1, activation='sigmoid'))
        
        img = layers.Input(shape=self.img_shape)
        validity = model(img)
        
        return models.Model(img, validity)
    
    def build_classifier(self):
        # Using the discriminator as a feature extractor
        feature_extractor = models.Sequential()
        
        # Copy layers from discriminator (excluding the final sigmoid layer)
        for i in range(len(self.discriminator.layers) - 1):
            feature_extractor.add(self.discriminator.layers[i])
            
        # Make these layers non-trainable
        for layer in feature_extractor.layers:
            layer.trainable = False
            
        # Add classification layers
        model = models.Sequential()
        model.add(feature_extractor)
        model.add(layers.Dense(128, activation='relu'))
        model.add(layers.Dropout(0.5))
        model.add(layers.Dense(1, activation='sigmoid'))
        
        return model
    
    def train_gan(self, X_train, epochs, batch_size=128):
        # Labels for real and fake images
        valid = np.ones((batch_size, 1))
        fake = np.zeros((batch_size, 1))
        
        for epoch in range(epochs):
            # Train Discriminator
            idx = np.random.randint(0, X_train.shape[0], batch_size)
            real_imgs = X_train[idx]
            
            noise = np.random.normal(0, 1, (batch_size, self.latent_dim))
            gen_imgs = self.generator.predict(noise, verbose=0)
            
            d_loss_real = self.discriminator.train_on_batch(real_imgs, valid)
            d_loss_fake = self.discriminator.train_on_batch(gen_imgs, fake)
            d_loss = 0.5 * np.add(d_loss_real, d_loss_fake)
            
            # Train Generator
            g_loss = self.combined.train_on_batch(noise, valid)
            
            if epoch % 200 == 0:
                print(f"Epoch {epoch}/{epochs} [D loss: {d_loss[0]:.4f}, acc.: {100*d_loss[1]:.2f}%] [G loss: {g_loss:.4f}]")
    
    def train_classifier(self, X_train, y_train, X_val, y_val, epochs=20, batch_size=32):
        # Train the classifier
        history = self.classifier.fit(
            X_train, y_train,
            batch_size=batch_size,
            epochs=epochs,
            validation_data=(X_val, y_val),
            verbose=1
        )
        
        return history
    
    def evaluate(self, X_test, y_test):
        # Predict on test set
        y_pred_prob = self.classifier.predict(X_test, verbose=0)
        y_pred = (y_pred_prob > 0.5).astype(int).flatten()
        
        # Calculate accuracy
        accuracy = accuracy_score(y_test, y_pred)
        
        # Print accuracy
        print(f"\nACCURACY: {accuracy:.4f} ({accuracy*100:.2f}%)")
        
        return accuracy

# Main execution
if __name__ == "__main__":
    # Parameters
    img_size = (128, 128)
    latent_dim = 100
    batch_size = 32
    gan_epochs = 1000
    classifier_epochs = 20
    
    # Generate synthetic data for demonstration
    # Replace this with your real data loading code
    print("Generating synthetic data...")
    X = np.random.rand(1000, img_size[0], img_size[1], 3) * 2 - 1  # Normalized to [-1, 1]
    y = np.random.randint(0, 2, size=(1000,))  # Binary labels (0: authentic, 1: forged)
    
    # Split data
    X_train, X_temp, y_train, y_temp = train_test_split(X, y, test_size=0.3, random_state=42)
    X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=0.5, random_state=42)
    
    # Initialize and train the GAN
    gan = ForgeryDetectionGAN(img_shape=(img_size[0], img_size[1], 3), latent_dim=latent_dim)
    
    print("Training GAN...")
    gan.train_gan(X_train, epochs=gan_epochs, batch_size=batch_size)
    
    print("\nTraining classifier...")
    gan.train_classifier(X_train, y_train, X_val, y_val, epochs=classifier_epochs, batch_size=batch_size)
    
    print("\nEvaluating model...")
    accuracy = gan.evaluate(X_test, y_test)